In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
params = list(net.parameters())
print(len(params))
print(params[0].size)

10
torch.Size([6, 1, 3, 3])


In [22]:
# random input 

input = torch.randn(1, 1, 32, 32)
print(input)
out = net(input)
print(out)


tensor([[[[-0.4190,  0.3030, -1.8026,  ..., -1.3835,  1.0316, -0.2742],
          [-0.6181,  1.8299, -0.3964,  ...,  0.0846,  0.5384,  0.0330],
          [ 0.7301,  0.2059,  0.4027,  ...,  1.9361, -1.2077,  0.5390],
          ...,
          [ 0.1578,  2.3675,  2.0379,  ..., -1.0102, -0.5842, -1.2781],
          [ 0.1060, -0.6504, -0.3409,  ...,  0.5930, -0.5079,  0.5631],
          [-2.3416,  0.6905, -1.1513,  ..., -1.1397, -0.2578, -0.4383]]]])
tensor([[-0.1150,  0.0276,  0.0131,  0.0413,  0.1146,  0.1011, -0.0628,  0.0629,
         -0.1714,  0.0061]], grad_fn=<AddmmBackward>)


In [23]:
# apply back propagation
net.zero_grad()
out.backward(torch.randn(1,10))

## Loss Function

In [24]:
output = net(input)
target = torch.randn(10) # a random true value
target = target.view(1, -1)
print(target)

criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)



tensor([[ 0.5659, -0.3530,  0.8665,  0.5279,  1.3138, -0.0908, -0.6715, -1.4805,
         -0.5655, -0.4392]])
tensor(0.6155, grad_fn=<MseLossBackward>)


In [25]:
print(loss.grad_fn)

In [26]:
# apply back propagation

net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0010,  0.0078,  0.0063, -0.0041,  0.0086, -0.0072])


## Update Weights

In [27]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [28]:
# More refined updating methods:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update